### Notebook 4: Training Mask R-CNN with ResNet-101 backbone

In [1]:
import os
from pathlib import Path
import cv2
import numpy as np
import pycocotools.mask as mask_util
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.structures import polygons_to_bitmask
setup_logger()

<Logger detectron2 (DEBUG)>

#### pretraining on LIVECell:

In [2]:
dataDir=Path('../LIVECell_dataset_2021/images/livecell_train_val_images')
cfg = get_cfg()
register_coco_instances('sartorius_train',{}, '../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json', dataDir)
register_coco_instances('sartorius_val',{},'../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_val.json', dataDir)
register_coco_instances('sartorius_test',{}, '../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_test.json', dataDir)

[01/09 15:56:04 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json takes 9.35 seconds.
[01/09 15:56:04 d2.data.datasets.coco]: Loaded 3253 images in COCO format from ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json


In [3]:
def polygon_to_rle(polygon, shape=(520, 704)):
    mask = polygons_to_bitmask([np.asarray(polygon) + 0.25], shape[0], shape[1])
    rle = mask_util.encode(np.asfortranarray(mask))
    return rle

def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1
    false_positives = np.sum(matches, axis=0) == 0 
    false_negatives = np.sum(matches, axis=1) == 0 
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred, targ):
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    enc_targs = [polygon_to_rle(enc_targ[0]) for enc_targ in enc_targs]
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)

class MAPIOUEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:
                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
        return {"MaP IoU": np.mean(self.scores)}

class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)

In [ ]:
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("sartorius_train", "sartorius_test")
cfg.DATASETS.TEST = ("sartorius_val",)
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0005
cfg.SOLVER.MAX_ITER = 100000
cfg.SOLVER.STEPS = []
cfg.SOLVER.CHECKPOINT_PERIOD = (len(DatasetCatalog.get('sartorius_train')) + len(DatasetCatalog.get('sartorius_test'))) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
cfg.TEST.EVAL_PERIOD = (len(DatasetCatalog.get('sartorius_train')) + len(DatasetCatalog.get('sartorius_test'))) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
print(cfg.OUTPUT_DIR)
trainer = Trainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[01/09 00:32:40 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json takes 8.99 seconds.
[01/09 00:32:40 d2.data.datasets.coco]: Loaded 3253 images in COCO format from ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json
[01/09 00:32:47 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_test.json takes 4.01 seconds.
[01/09 00:32:47 d2.data.datasets.coco]: Loaded 1564 images in COCO format from ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_test.json
[01/09 00:32:59 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json takes 9.49 seconds.
[01/09 00:32:59 d2.data.datasets.coco]: Loaded 3253 images in COCO format from ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json
[01/09 00:33:07 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_test.json takes 4.02 second

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (9, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (9,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (32, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (32,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (8, 256, 1, 1) 

[01/09 00:33:32 d2.engine.train_loop]: Starting training from iteration 0


/home/hamdi/miniconda3/envs/env/lib/python3.9/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/home/hamdi/miniconda3/envs/env/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272204863/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[01/09 00:33:39 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f3ceb732160> to CPU due to CUDA OOM
[01/09 00:33:45 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f3ceb732160> to CPU due to CUDA OOM
[01/09 00:33:49 d2.utils.events]:  eta: 12:35:51  iter: 19  total_loss: 8.775  loss_cls: 2.448  loss_box_reg: 0.335  loss_mask: 0.6909  loss_rpn_cls: 4.796  loss_rpn_loc: 0.3782  time: 0.8019  data_time: 0.0237  lr: 9.9905e-06  max_mem: 8260M
[01/09 00:33:56 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f3ceb732160> to CPU due to CUDA OOM
[01/09 00:34:01 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f3ceb732160> to CPU due to CUDA OOM
[01/09 00:34:05 d2.utils.events]:  eta: 12:46:02  iter: 39  total_loss: 4.431  loss_cls: 2.083  loss_box_reg: 0.4285  loss_mask: 0.6847  loss_rpn_cls: 0.9408  loss_rpn_loc: 0.2961  time: 0.7969  data_time: 0.0067  lr: 1.998e-05  max_mem: 8694M
[

In [ ]:
# resume training after crash
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("sartorius_train", "sartorius_test")
cfg.DATASETS.TEST = ("sartorius_val",)
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0005
cfg.SOLVER.MAX_ITER = 100000
cfg.SOLVER.STEPS = []
cfg.SOLVER.CHECKPOINT_PERIOD = (len(DatasetCatalog.get('sartorius_train')) + len(DatasetCatalog.get('sartorius_test'))) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
cfg.TEST.EVAL_PERIOD = (len(DatasetCatalog.get('sartorius_train')) + len(DatasetCatalog.get('sartorius_test'))) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
print(cfg.OUTPUT_DIR)
trainer = Trainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

[01/09 10:10:34 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json takes 8.37 seconds.
[01/09 10:10:34 d2.data.datasets.coco]: Loaded 3253 images in COCO format from ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json
[01/09 10:10:42 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_test.json takes 4.79 seconds.
[01/09 10:10:42 d2.data.datasets.coco]: Loaded 1564 images in COCO format from ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_test.json
[01/09 10:10:53 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json takes 9.20 seconds.
[01/09 10:10:53 d2.data.datasets.coco]: Loaded 3253 images in COCO format from ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json
[01/09 10:11:00 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_test.json takes 3.95 second

/home/hamdi/miniconda3/envs/env/lib/python3.9/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/home/hamdi/miniconda3/envs/env/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272204863/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[01/09 10:11:29 d2.utils.events]:  eta: 5:30:25  iter: 52979  total_loss: 1.348  loss_cls: 0.308  loss_box_reg: 0.4504  loss_mask: 0.2859  loss_rpn_cls: 0.07957  loss_rpn_loc: 0.1901  time: 0.4217  data_time: 0.0818  lr: 0.0005  max_mem: 4770M
[01/09 10:11:30 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f2655572160> to CPU due to CUDA OOM
[01/09 10:11:39 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7f2655572160> to CPU due to CUDA OOM
[01/09 10:11:44 d2.utils.events]:  eta: 5:49:52  iter: 52999  total_loss: 1.104  loss_cls: 0.2429  loss_box_reg: 0.3797  loss_mask: 0.2582  loss_rpn_cls: 0.06567  loss_rpn_loc: 0.1562  time: 0.7160  data_time: 0.0060  lr: 0.0005  max_mem: 8277M
[01/09 10:11:54 d2.utils.events]:  eta: 5:55:30  iter: 53019  total_loss: 1.254  loss_cls: 0.3219  loss_box_reg: 0.3736  loss_mask: 0.2724  loss_rpn_cls: 0.07786  loss_rpn_loc: 0.1773  time: 0.6040  data_time: 0.0069  lr: 0.0005  max_mem: 8277M
[01/09 10:

In [4]:
# resume training
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("sartorius_train", "sartorius_test")
cfg.DATASETS.TEST = ("sartorius_val",)
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0005
cfg.SOLVER.MAX_ITER = 100000
cfg.SOLVER.STEPS = []
cfg.SOLVER.CHECKPOINT_PERIOD = (len(DatasetCatalog.get('sartorius_train')) + len(DatasetCatalog.get('sartorius_test'))) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
cfg.TEST.EVAL_PERIOD = (len(DatasetCatalog.get('sartorius_train')) + len(DatasetCatalog.get('sartorius_test'))) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
print(cfg.OUTPUT_DIR)
trainer = Trainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()
os.rename("output", "output_4.1")

[01/09 15:56:26 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json takes 9.14 seconds.
[01/09 15:56:26 d2.data.datasets.coco]: Loaded 3253 images in COCO format from ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json
[01/09 15:56:33 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_test.json takes 4.02 seconds.
[01/09 15:56:33 d2.data.datasets.coco]: Loaded 1564 images in COCO format from ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_test.json
[01/09 15:56:45 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json takes 9.61 seconds.
[01/09 15:56:45 d2.data.datasets.coco]: Loaded 3253 images in COCO format from ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train.json
[01/09 15:56:53 d2.data.datasets.coco]: Loading ../LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_test.json takes 4.15 second

/home/hamdi/miniconda3/envs/env/lib/python3.9/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/home/hamdi/miniconda3/envs/env/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272204863/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[01/09 15:57:24 d2.utils.events]:  eta: 2:30:28  iter: 81879  total_loss: 1.16  loss_cls: 0.2493  loss_box_reg: 0.366  loss_mask: 0.253  loss_rpn_cls: 0.0808  loss_rpn_loc: 0.1625  time: 0.5244  data_time: 0.0460  lr: 0.0005  max_mem: 6625M
[01/09 15:57:25 d2.utils.memory]: Attempting to copy inputs of <function pairwise_iou at 0x7fa42d791160> to CPU due to CUDA OOM
[01/09 15:57:36 d2.utils.events]:  eta: 2:22:26  iter: 81899  total_loss: 1.075  loss_cls: 0.2345  loss_box_reg: 0.3454  loss_mask: 0.2683  loss_rpn_cls: 0.07507  loss_rpn_loc: 0.1411  time: 0.5824  data_time: 0.0055  lr: 0.0005  max_mem: 7962M
[01/09 15:57:46 d2.utils.events]:  eta: 2:23:13  iter: 81919  total_loss: 1.016  loss_cls: 0.2301  loss_box_reg: 0.3285  loss_mask: 0.2498  loss_rpn_cls: 0.05571  loss_rpn_loc: 0.1403  time: 0.5432  data_time: 0.0055  lr: 0.0005  max_mem: 7962M
[01/09 15:57:56 d2.utils.events]:  eta: 2:23:36  iter: 81939  total_loss: 1.247  loss_cls: 0.2757  loss_box_reg: 0.4174  loss_mask: 0.2939  l

Lowest validation mAP is obtained after iteration 72239

Now we train on the competition data

In [3]:
dataDir=Path('../')
cfg = get_cfg()
cfg.INPUT.MASK_FORMAT='bitmask'
register_coco_instances('sartorius_train',{}, '../sartorius-annotations-coco-format/annotations_train.json', dataDir)
register_coco_instances('sartorius_val',{},'../sartorius-annotations-coco-format/annotations_val.json', dataDir)

[01/09 20:33:38 d2.data.datasets.coco]: Loaded 485 images in COCO format from ../sartorius-annotations-coco-format/annotations_train.json


In [2]:
def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Missed objects
    false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred, targ):
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)

class MAPIOUEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:
                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
        return {"MaP IoU": np.mean(self.scores)}

class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)

In [4]:
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("sartorius_train",)
cfg.DATASETS.TEST = ("sartorius_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = 'output_4.1/model_0072239.pth'
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0005
cfg.SOLVER.MAX_ITER = 10000
cfg.SOLVER.STEPS = []
cfg.SOLVER.CHECKPOINT_PERIOD = len(DatasetCatalog.get('sartorius_train')) // cfg.SOLVER.IMS_PER_BATCH
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
cfg.TEST.EVAL_PERIOD = len(DatasetCatalog.get('sartorius_train')) // cfg.SOLVER.IMS_PER_BATCH

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = Trainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[01/09 20:33:47 d2.data.datasets.coco]: Loaded 485 images in COCO format from ../sartorius-annotations-coco-format/annotations_train.json
[01/09 20:33:48 d2.data.datasets.coco]: Loaded 485 images in COCO format from ../sartorius-annotations-coco-format/annotations_train.json
[01/09 20:33:52 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (t

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (9, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (9,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (32, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (32,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (8, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) in th

[01/09 20:33:53 d2.engine.train_loop]: Starting training from iteration 0


/home/hamdi/miniconda3/envs/env/lib/python3.9/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/home/hamdi/miniconda3/envs/env/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272204863/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[01/09 20:34:07 d2.utils.events]:  eta: 1:17:40  iter: 19  total_loss: 3.164  loss_cls: 1.409  loss_box_reg: 0.5271  loss_mask: 0.6927  loss_rpn_cls: 0.2986  loss_rpn_loc: 0.2472  time: 0.6627  data_time: 0.2516  lr: 9.9905e-06  max_mem: 5665M
[01/09 20:34:23 d2.utils.events]:  eta: 1:19:02  iter: 39  total_loss: 3.027  loss_cls: 1.31  loss_box_reg: 0.3892  loss_mask: 0.6913  loss_rpn_cls: 0.3138  loss_rpn_loc: 0.2762  time: 0.7167  data_time: 0.3190  lr: 1.998e-05  max_mem: 5965M
[01/09 20:34:36 d2.utils.events]:  eta: 1:18:55  iter: 59  total_loss: 2.761  loss_cls: 1.148  loss_box_reg: 0.4739  loss_mask: 0.6846  loss_rpn_cls: 0.2243  loss_rpn_loc: 0.2476  time: 0.6995  data_time: 0.2216  lr: 2.997e-05  max_mem: 5965M
[01/09 20:34:46 d2.utils.events]:  eta: 1:16:49  iter: 79  total_loss: 2.57  loss_cls: 0.8989  loss_box_reg: 0.5694  loss_mask: 0.6737  loss_rpn_cls: 0.1967  loss_rpn_loc: 0.2339  time: 0.6440  data_time: 0.0554  lr: 3.9961e-05  max_mem: 5965M
[01/09 20:34:58 d2.utils.ev

Best validation IoU = 0.2725 obtained after iteration 8469

In [5]:
%load_ext tensorboard

In [7]:
%tensorboard --logdir output_4.2